# Introduction

This notebook is all about training a YOLO model to detect handwritten characters from the synthetically generated training dataset.

In [1]:
# You'll need to install the following libraries to run this notebook
!pip install numpy matplotlib opencv-python seaborn pandas tqdm keras tensorflow

In [2]:
# Setup all imports here
from collections import Counter
import random
from time import time
import os
from glob import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from tqdm.notebook import trange, tqdm_notebook

In [3]:
# Show all matplotlib plots without always calling plt.show()
%matplotlib inline

In [4]:
# Increase the size of the matplotlib figures
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)

# Training the model

YOLO stands for You Only Look Once. It's a relatively novel neural network architecture that allows pretty fast inferencing while keeping a respectible amount of accuracy.

* Original YOLO Paper: https://arxiv.org/pdf/1506.02640.pdf
* YOLO v3 Paper: https://pjreddie.com/media/files/papers/YOLOv3.pdf

In [6]:
# Install the Keras version of YOLO from 'Borda'
!git clone https://github.com/wizyoung/YOLOv3_TensorFlow.git yolov3-tf

Cloning into 'yolov3-tf'...
remote: Enumerating objects: 222, done.
remote: Total 222 (delta 0), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (222/222), 3.16 MiB | 3.46 MiB/s, done.
Resolving deltas: 100% (127/127), done.
python: can't open file 'setup.py': [Errno 2] No such file or directory


## Testing YOLO with COCO

In [15]:
# Download the YOLO v3 weights file
!wget https://pjreddie.com/media/files/yolov3.weights \
    --output-document='./yolov3-tf/data/darknet_weights/yolov3.weights' \
    --quiet --show-progress

eights/yolov3.weigh  11%[=>                  ]  27.55M  5.17MB/s    eta 58s    ^C


In [18]:
# Convert the COCO-pretained model weights
!cd yolov3-tf && python convert_weight.py

W0212 15:52:39.867549 139866816436032 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0212 15:52:40.190120 139866816436032 module_wrapper.py:139] From convert_weight.py:22: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-02-12 15:52:40.195695: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tejashah88/extra-software/muse-sdk
2020-02-12 15:52:40.196031: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2020-02-

In [21]:
!ls yolov3-tf/data/demo_data

dog.jpg  kite.jpg  messi.jpg  results/


In [26]:
# NOTE: Don't close the GUI window by clicking the 'X'; press any key on the keyboard instead
# Otherwise this cell will keep on running and you'll have to interrupt the kernel
!cd yolov3-tf && python test_single_image.py ./data/demo_data/kite.jpg

W0212 23:46:18.800108 140642383140672 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0212 23:46:19.182229 140642383140672 module_wrapper.py:139] From test_single_image.py:48: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-02-12 23:46:19.183091: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tejashah88/extra-software/muse-sdk
2020-02-12 23:46:19.183110: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2020-

## Training a custom model